# Langchain Testing Notebook
In order to get back up to speed with Langchain and all of its components, I'm going to be writing a series of notebooks that will test the functionality of each component. This notebook will test the functionality of using Langchain with our Necromunda dataset and see if we can get it to simulate a match

## Imports and Config

In [97]:
from langchain_openai import ChatOpenAI, OpenAI, OpenAIEmbeddings
import openai
from langchain.adapters import openai as lc_openai
from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
from langchain.indexes import index
from langchain.indexes.base import RecordManager
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

from langchain import hub
from langchain.agents import create_openai_functions_agent, load_huggingface_tool, load_tools
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os 

TavilySearchResults.tavily_api_key = TAVILY_API_KEY

import yaml
from langchain.agents import create_json_agent
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec

from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool


from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

from langchain.schema import HumanMessage, SystemMessage
python_tool = [PythonREPLTool()]

requests_tools = load_tools(["requests_all"])
from langchain_community.utilities import TextRequestsWrapper

requests = TextRequestsWrapper()

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI, OpenAI


## Necromunda Schemas

In [98]:
weapon_schema = {
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"},
        "damage": {"type": "integer"},
        "range": {"type": "integer"},
        "ammo": {"type": "integer"},
        "special_ability": {"type": "string"},
    },
    "required": ["name", "type", "damage", "range", "ammo"],
}

ganger_schema = {
    "properties": {
        "name": {"type": "string"},
        "points": {"type": "integer"},
        "faction": {"type": "string"},
        "weapons": {"type": "arrau", "items": {"type": "string"}},
        "armor": {"type": "string"},
        "skills": {"type": "array", "items": {"type": "string"}},
    },
    "required": ["name", "height", "gang", "weapon", "armor"],
}
gang_schema = {
    "properties": {
        "name": {"type": "string"},
        "leader": {"type": "string"},
        "territory": {"type": "string"},
        "members": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "height": {"type": "integer"},
                    "hair_color": {"type": "string"},
                    "weapon": {"type": "string"},
                    "armor": {"type": "string"},
                    "skills": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["name", "height", "weapon", "armor"],
            },
        },
    },
    "required": ["name", "leader", "territory", "members"],
}

In [99]:
chat = ChatOpenAI()

In [2]:
%pip install d20

Note: you may need to restart the kernel to use updated packages.Collecting d20
  Using cached d20-1.1.2-py3-none-any.whl (20 kB)
  Using cached lark-parser-0.9.0.tar.gz (298 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lark-parser: filename=lark_parser-0.9.0-py2.py3-none-any.whl size=77202 sha256=43725cf5156e4221a9865cd96773857f031590496e442aaee10426384a8e5527
  Stored in directory: c:\users\dcarm\appdata\local\pip\cache\wheels\72\b3\c7\0482ee4f592faa0b1941cf06346e0e683856812a1c7daa2e9b
Successfully built lark-parser




[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import d20

# basic dice rolling function
def roll_dice(num_dice, dice_type):
    rolls = d20.roll(f"{num_dice}d{dice_type}")
    return rolls.total

# roll for necromunda attacks
def roll_attack(num_dice, dice_type, hit_roll, wound_roll, armor_save):
    hits = 0
    for i in range(num_dice):
        roll = roll_dice(1, dice_type)
        if roll >= hit_roll:
            wound = roll_dice(1, dice_type)
            if wound >= wound_roll:
                armor = roll_dice(1, dice_type)
                if armor < armor_save:
                    hits += 1
    return hits

# roll for necromunda injuries
def roll_injury(num_dice, dice_type, injury_roll):
    injuries = 0
    for i in range(num_dice):
        roll = roll_dice(1, dice_type)
        if roll >= injury_roll:
            injuries += 1
    return injuries




In [100]:
messages = chat(
    [
        HumanMessage(
            content="I would like to create a 1000 point Necromunda gang using 1000 points for the Cawdor faction.  Make sure to include stats and equipment for each member."
        )
    ]
)

c:\Users\dcarm\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Sure! Here\'s a suggested 1000 point Necromunda gang for the Cawdor faction:\n\nLeader:\n- Name: Lazarus the Zealot\n- Cost: 180 points\n- Type: Heavy\n- Movement: 4"\n- Weapon: Heavy Crossbow (18", S4, AP-1, D1)\n- Equipment: Flak Armor, Smoke Grenades, Melta Bomb\n- Abilities: Overseer, Leader, Fanatical Devotion\n\nChampion 1:\n- Name: Ezekiel the Purifier\n- Cost: 160 points\n- Type: Ganger\n- Movement: 4"\n- Weapon: Heavy Flamer (Template, S5, AP-1, D1)\n- Equipment: Flak Armor, Smoke Grenades, Frag Grenades\n- Abilities: Overseer, Nerves of Steel\n\nChampion 2:\n- Name: Miriam the Redeemer\n- Cost: 150 points\n- Type: Ganger\n- Movement: 4"\n- Weapon: Heavy Stubber (36", S4, AP0, D1)\n- Equipment: Flak Armor, Smoke Grenades, Frag Grenades\n- Abilities: Overseer, Step Aside\n\nGanger 1:\n- Name: Samuel the Penitent\n- Cost: 100 points\n- Type: Ganger\n- Movement: 4"\n- Weapon: Autogun (24", S3, AP0, D1)\n- Equipment: Flak Armor, Smoke Grenades\n- Abilities: Over

In [ ]:



llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)
tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [88]:
requests.get("https://necrovox.org/docs/gangs/gang-lists/palanite-enforcers/")

'<!doctype html>\n<html lang="en" dir="ltr" class="docs-wrapper plugin-docs plugin-id-default docs-version-current docs-doc-page docs-doc-id-gangs/gang-lists/palanite-enforcers/palanite-enforcers" data-has-hydrated="false">\n<head>\n<meta charset="UTF-8">\n<meta name="generator" content="Docusaurus v3.0.1">\n<title data-rh="true">Palanite Enforcers | Necro-vox</title><meta data-rh="true" name="viewport" content="width=device-width,initial-scale=1"><meta data-rh="true" name="twitter:card" content="summary_large_image"><meta data-rh="true" property="og:url" content="https://necrovox.org/docs/gangs/gang-lists/palanite-enforcers/"><meta data-rh="true" property="og:locale" content="en"><meta data-rh="true" name="docusaurus_locale" content="en"><meta data-rh="true" name="docsearch:language" content="en"><meta data-rh="true" name="author" content="servo_scribe"><meta data-rh="true" name="docusaurus_version" content="current"><meta data-rh="true" name="docusaurus_tag" content="docs-default-cur

In [ ]:




tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
# import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish


# Define the agent
# Note that here, we are using `.assign` to add the output of the agent to the dictionary
# This dictionary will be returned from the node
# The reason we don't want to return just the result of `agent_runnable` from this node is
# that we want to continue passing around all the other inputs
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data.pop('agent_outcome')
    # Get the tool to use
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    # We now add in the action and the observation to the `intermediate_steps` list
    # This is the list of all previous actions taken and their output
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, Graph

workflow = Graph()

# Add the agent node, we give it name `agent` which we will use later
workflow.add_node("agent", agent)
# Add the tools node, we give it name `tools` which we will use later
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "exit": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('tools', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
chain = workflow.compile()

In [ ]:
chain.invoke({"input": "what is the weather in sf", "intermediate_steps": []})

In [ ]:
%pip install unstructured markdown langgraph

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import markdown
markdown_path = ".\\necrovox_docs\\rules.md"
loader = UnstructuredMarkdownLoader(markdown_path, mode="elements")
data = loader.load()

In [ ]:
data

In [ ]:
embedding_function = OpenAIEmbeddings()

In [ ]:
import chromadb

#  = chromadb.PersistentClient()
persistent_client = chromadb.PersistentClient(path="./necromunda_db/")
collection = persistent_client.get_or_create_collection("necromunda")
vector_store = Chroma(
    client=persistent_client,
    collection_name="necromunda",
    embedding_function=embedding_function,
)
# docs = db2.similarity_search(query)

In [ ]:


print("There are", vector_store._collection.count(), "in the collection")

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr")

In [ ]:
from 

In [ ]:
query = "How can I make an injury roll in Necromunda?"

In [ ]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
local_store = LocalFileStore("./cache/")


cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding_function, local_store, namespace=embedding_function.model
)

In [ ]:
list(local_store.yield_keys())

In [ ]:

vector_store.similarity_search_with_score(query)

In [ ]:
retriever.get_relevant_documents(query)[0]

In [ ]:

from langchain.retrievers import SelfQueryRetriever
# from langchain.chain import Chain
retriever2 = SelfQueryRetriever(langchain_chroma)

In [ ]:
retriever = langchain_chroma.as_retriever()
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [ ]:
from operator import itemgetter

# from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.get_prompts()

In [ ]:
# Reduce the length of the messages
chain = chain | RunnableLambda(lambda x: x[:200])

chain.invoke(query)